In [1]:
# 1. Lemmatizer 객체를 생성하고 아래 단어 리스트에 대해 표제어를 추출해보고 전 후 단어를 비교하시오
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

words = ['policy', 'doing', 'organization', 'have', 'going', 'love', 'lives', 'fly', 'dies', 'watched', 'has', 'starting']

print('표제어 추출 전 :',words)
print('표제어 추출 후 :',[lemmatizer.lemmatize(word) for word in words])

#  dy나 ha와 같이 의미를 알 수 없는 적절하지 못한 단어를 출력됨
# 이는 표제어 추출기(lemmatizer)가 본래 단어의 품사 정보를 알아야만 정확한 결과를 얻을 수 있기 때문

표제어 추출 전 : ['policy', 'doing', 'organization', 'have', 'going', 'love', 'lives', 'fly', 'dies', 'watched', 'has', 'starting']
표제어 추출 후 : ['policy', 'doing', 'organization', 'have', 'going', 'love', 'life', 'fly', 'dy', 'watched', 'ha', 'starting']


In [3]:
# 2. dies, watched, has 에 대해 표제어 추출 시 품사정보를 입력하여 결과를 확인하시오
print(lemmatizer.lemmatize('dies', 'v'))
print(lemmatizer.lemmatize('watched', 'v'))
print(lemmatizer.lemmatize('has', 'v'))

die
watch
have


In [4]:
# 3. Stemmer 객체를 생성하고 아래 단어 리스트에 대해 어간을 추출해보고 전 후 단어를 비교하시오
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
words = ['formalize', 'allowance', 'electricical']

print('어간 추출 전 :',words)
print('어간 추출 후 :',[stemmer.stem(word) for word in words])

어간 추출 전 : ['formalize', 'allowance', 'electricical']
어간 추출 후 : ['formal', 'allow', 'electric']


- 어간 추출 속도 표제어 추출보다 일반적으로 빠름
- 포터 어간 추출기는 정밀하게 설계되어 정확도가 높으므로 영어 자연어 처리에서 어간 추출을 하고자 한다면 가장 준수한 선택임

In [5]:
# 4. 아래 동일한 단어 리스트에 대해 표제어 추출과 어간 추출을 수행해보고 차이를 확인하시오
words = ['am', 'the going', 'having']

print('원래 단어 :',words)
print('표제어 추출 후 :',[lemmatizer.lemmatize(word) for word in words])
print('어간 추출 후 :',[stemmer.stem(word) for word in words])

원래 단어 : ['am', 'the going', 'having']
표제어 추출 후 : ['am', 'the going', 'having']
어간 추출 후 : ['am', 'the go', 'have']


#### 구남이

In [9]:
import pandas as pd

twit_df = pd.read_csv('./data/Twitter_Data.csv', header=0, sep=",")
twit_df.head(3)
twit_df['category'].value_counts()

# 이진 분류 문제로 만들기 위해 category가 -1인 경우 삭제
index = twit_df[twit_df['category']==-1].index
twit_df.drop(index, inplace=True)

In [10]:
# 1. 결측치가 있는지 확인해보기
twit_df.isnull().sum()

clean_text    3
category      7
dtype: int64

In [15]:
# 2. 결측치가 있는 행 삭제하기
twit_df.dropna(inplace=True)

In [18]:
import re
# 3. 파이썬의 정규 표현식 모듈인 re를 이용하여 영어 문자열이 아닌 문자는 모두 공백으로 변환 
twit_df['clean_text'] = twit_df['clean_text'].apply( lambda x : re.sub('[^a-zA-Z]'," ",x) )

In [19]:
from sklearn.model_selection import train_test_split

# 결정값 클래스인 category 칼럼을 별도로 추출해 결정 값 데이터 세트를 만들기
class_df = twit_df['category']
# 원본 데이터 세트에서 category 칼럼을 삭제해 피처 데이터 세트를 생성
feature_df = twit_df.drop(['category'], axis=1, inplace=False)

# rain_test_split()을 이용해 학습용과 테스트용 데이터 세트로 분리
X_train, X_test, y_train, y_test= train_test_split(feature_df, class_df, test_size=0.3, random_state=156)

X_train.shape, X_test.shape

((89222, 1), (38238, 1))

In [24]:
# clean_text텍스트를 Count 벡터화를 이용해 피처 벡터화
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

# 5. 
# 스톱 워드는 English, filtering, ngram은 (1,2)로 설정해 CountVectorization수행. 
# LogisticRegression의 sorlver='liblinear', C는 10으로 설정. 
pipeline = Pipeline([
    ('cnt_vect', CountVectorizer(ngram_range=(1,2), stop_words = 'english')),
    ('lr_clf', LogisticRegression(solver='liblinear',C=10))])

In [26]:
# 6. Pipeline 객체를 이용하여 학습/예측 수행 
pipeline.fit(X_train['clean_text'], y_train)
pred = pipeline.predict(X_test['clean_text'])
pred_probs = pipeline.predict_proba(X_test['clean_text'])[:,1]

print('예측 정확도는 {0:.4f}, ROC-AUC는 {1:.4f}'.format(accuracy_score(y_test ,pred),
                                         roc_auc_score(y_test, pred_probs)))

예측 정확도는 0.9122, ROC-AUC는 0.9563


#### 강수민

In [27]:
# Q1. 코사인 유사도 구하는 함수 작성
import numpy as np

def cos_similarity(v1, v2):
    dot_product = np.dot(v1,v2)
    l2_norm = (np.sqrt(sum(np.square(v1))) * np.sqrt(sum(np.square(v2))))
    similarity = dot_product / l2_norm     
    
    return similarity

In [29]:
# Q2. 문서를 TF-IDF로 벡터화된 행렬로 변환하기
from sklearn.feature_extraction.text import TfidfVectorizer

doc_list = ['if you take the blue pill, the story ends' ,
            'if you take the red pill, you stay in Wonderland',
            'if you take the red pill, I show you how deep the rabbit hole goes']

tfidf_vect_simple = TfidfVectorizer()
feature_vect_simple = tfidf_vect_simple.fit_transform(doc_list)

In [31]:
# Q3. feature_vect_simple 행렬을 Dense Matrix로 변환하기 
feature_vect_dense = feature_vect_simple.todense()

vect1 = np.array(feature_vect_dense[0]).reshape(-1,)
vect2 = np.array(feature_vect_dense[1]).reshape(-1,)
vect3 = np.array(feature_vect_dense[2]).reshape(-1,)

# 두번째 문장과 세번째 문장의 feature vector로 두개 문장의 Cosine 유사도 추출
similarity_simple = cos_similarity(vect2, vect3)
print('문장 2, 문장 3 Cosine 유사도: {0:.3f}'.format(similarity_simple))

문장 2, 문장 3 Cosine 유사도: 0.456


In [32]:
# Q4. 사이킷런의 cosine_similarity로 확인하기(ndarray 형태로)
from sklearn.metrics.pairwise import cosine_similarity

similarity_simple_pair = cosine_similarity(feature_vect_simple[0],feature_vect_simple[1:])
print(similarity_simple_pair)

[[0.40207758 0.40425045]]


#### 박민영

In [36]:
# 1. 1986년에 개봉한 영화 topgun의 리뷰이다. 
# 아래 데이터프레임을 로딩한 후 review와 임곗값 0.1을 입력으로 받아 감성분석 결과를 내는 함수 vader_polarity를 생성하라.

import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer

review_df = pd.DataFrame({'id':[1, 2, 3, 4, 5, 6],
                          'review':['It is an interesting movie however the movie takes God name in vain and has coarse language. My wife and I use TV guardian and filtered out the cuss words otherwise I would not have watched it. Also there is a sexual scene between 2 unmarried people. The movie itself was good however without the cuss word filter I would not have watched it.',
                                   'After making a few mistakes, Maverick and the others are forced to "perfect" their flights, thereby showing viewers the importance of attention to detail and getting things right, and the consequences when those are not heeded. Competition and the desire to win are shown, but so is teamwork. A healthy balance is found. When Maverick meets Charlie he changes for the better, reinforcing the idea that meeting someone special sometimes makes us want to change our behavior. While you could argue that there shouldve been a couple women pilots, Charlie, a civilian working with the military, is strong, which more than makes up for that. As for the smoking, it wasnt frowned upon in 1986 to the degree that it is today, especially in the military. While parts of this movie will go over viewers heads unless either they are in or have been in the military, everyone can benefit from this movie.',
                                   'Anybody could see it Kids upto adults',
                                   'Amazing Navy fighter pilot movie; touches on the risks of being a fighter pilot. There werent that many scary scenes. A Navy pilot goes into a daze and nearly crashes into the ocean. In a training accident, a major character is killed when their ejection malfunctions, which is sad. Otherwise, there is a lot of swearing, and missiles, and explosions, and a man flicks an enemy pilot off in mid-air. The worst thing was the scene where they made out, which I would recommend closing your eyes for.',
                                   'I just dont understand why this movie is so highly rated. They could save it with top gun maverick, just like karate kid (another overrated, boring film) did with cobra kai. There is a sex scene, and if you pay attention, you can see a woman take off her shirt, and reveal her breasts to Maverick (no nudity, just upper heads). The reason this got 2 stars is because they did a good job with the stunts, if it wasnt for that, this would be at 1.',
                                   'I dont really care for Tom cruise movies. I always think his movies gonna be good but it isn’t as good as I thought'],
                          'sentiment':[0, 1, 1, 1, 0, 0]})

def vader_polarity(review, threshold = 0.1):
    analyzer = SentimentIntensityAnalyzer()
    scores = analyzer.polarity_scores(review)
    
    agg_scores = scores['compound']
    final_sentiment = 1 if agg_scores >= threshold else 0
    return final_sentiment

In [37]:
# 2. apply lambda 식을 이용해 레코드별로 vader_polarity()를 수행하고 결과를 vader_preds에 저장하여라.

from nltk.sentiment.vader import SentimentIntensityAnalyzer
review_df['vader_preds'] = review_df['review'].apply(lambda x : vader_polarity(x,0.1))

In [39]:
# 3. 감성분석 결과를 이용하여 정확도, 정밀도, 재현율을 측정하여라.

from sklearn.metrics import accuracy_score, precision_score, recall_score
import numpy as np
y_target = review_df['sentiment'].values
vader_preds = review_df['vader_preds'].values

print('정확도 :', np.round(accuracy_score(y_target, vader_preds),4))
print('정밀도 :', np.round(precision_score(y_target, vader_preds),4))
print('재현율 :', np.round(recall_score(y_target, vader_preds),4))

정확도 : 0.1667
정밀도 : 0.25
재현율 : 0.3333
